## What relationships exist between type of admission, admission diagnosis, length of stay, age, gender, readmittance within 30 days, amount of bedside attention (e.g. patient is bathed), and occurence of sepsis?

EDA: Pairplot with every pairing of the eight attributes mentioned above. The data from ADMISSIONS, PATIENTS, and CHARTEVENTS would have to be merged and derived attributes added. The data would be primarily segmented by ADMISSIONS record. Visualization: Seaborn.

In [1]:
# Imports for DFs & connecting to Postgres
import pandas as pd
import psycopg2

- type of admission -> admissions.admission_type
- admission diagnosis -> admissions.diagnosis
- length of stay -> admissions.dischtime minus admissions.admittime
- readmittance within 30 days -> admissions.admittime? (will have to calculate for each admission whether another admission took place within 30 days)
- age -> patients.dob (will have to calculate age at time of admission)
- gender -> patients.gender
- amount of bedside attention (e.g. patient is bathed) -> chartevents.itemid.label
- occurence of sepsis -> 

### Pull in Admissions data

In [2]:
# Connect to Postgres & get all records for ADMISSIONS
try:
    con = psycopg2.connect("host='localhost' dbname='mimic' user='postgres' password='postgres'")
    cur = con.cursor()
    cur.execute ("""SELECT * FROM mimiciii.admissions;""")
    con.commit()
    print('OK')
except Exception as e:
    print(e)  

OK


In [3]:
# Store ADMISSIONS result in var
admissions_all = cur.fetchall()

In [5]:
# Convert ADMISSIONS result to DF
admissions_df = pd.DataFrame(admissions_all, columns = ['row_id','subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
 'admission_type', 'admission_location', 'discharge_location',
 'insurance', 'language', 'religion', 'marital_status', 'ethnicity',
 'edregtime', 'edouttime', 'diagnosis', 'hospital_expire_flag',
 'has_chartevents_data'])

In [15]:
# Create shortened DF of relevant cols
admissions_short = admissions_df[['subject_id', 'hadm_id', 'admission_type', 'diagnosis', 'dischtime', 'admittime']]

In [28]:
admissions_short.shape

(58976, 6)

### Pull in Patients data

In [18]:
# Connect to Postgres & get all records for PATIENTS
try:
    con = psycopg2.connect("host='localhost' dbname='mimic' user='postgres' password='postgres'")
    cur = con.cursor()
    cur.execute ("""SELECT * FROM mimiciii.patients;""")
    con.commit()
    print('OK')
except Exception as e:
    print(e)

OK


In [19]:
# Store PATIENTS result in var
patients_all = cur.fetchall()

In [20]:
# Convert PATIENTS result to DF
patients_df = pd.DataFrame(patients_all, columns = ['row_id', 'subject_id', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag'])

In [26]:
# Create shortened DF of relevant cols
patients_short = patients_df[['subject_id', 'gender', 'dob']]

In [27]:
patients_short.shape

(46520, 3)

### Merge Patient details onto Admissions

In [29]:
adm_pat_merge = admissions_short.merge(patients_short, how='left', on='subject_id')

In [33]:
adm_pat_merge.sample(3)

,subject_id,hadm_id,admission_type,diagnosis,dischtime,admittime,gender,dob
58548,96659,144266,EMERGENCY,ASTHMA;CHRONIC OBST PULM DISEASE,2156-04-30 17:00:00,2156-04-26 14:19:00,F,2082-02-09
6802,4489,175073,EMERGENCY,PNEUMONIA,2159-08-13 14:37:00,2159-07-30 19:17:00,M,2130-02-24
46201,54891,189389,ELECTIVE,"MITRAL STENOSIS\MITRAL VALVE REPAIR, ? REPLACE...",2166-01-21 13:22:00,2166-01-13 13:15:00,M,2086-03-07


In [46]:
# adm_pat_merge['dischtime'][0]

In [45]:
# adm_pat_merge['dischtime'][0].hour

In [48]:
adm_pat_merge['adm_los'] = adm_pat_merge['dischtime']-adm_pat_merge['admittime']

In [ ]:
adm_

In [59]:
adm_pat_merge['adm_los'][0].seconds

12480

In [60]:
adm_pat_merge['adm_los'][0].days

1